# **PRÉ-PROCESSAMENTO DOS DADOS**

In [1]:
import numpy as np
import pandas as pd
import re

In [63]:
df = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_raw.csv')
df.head()

,ID,Peso,Altura,IMC,Atendimento,DN,IDADE,Convenio,PULSOS,PA SISTOLICA,...,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
0,1,5.0,51,19.0,11/05/06,30/03/06,0.12,GS,Normais,NaN,...,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,2,3.5,50,14.0,25/05/05,19/05/05,0.02,GS,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia
2,3,0.0,0,NaN,12/06/01,08/05/05,-4.05,SULA,Normais,NaN,...,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN
3,4,8.1,65,19.0,15/10/09,21/04/09,0.50,NaN,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN
4,7,40.0,151,18.0,14/01/08,14/08/95,12.89,SAME,Normais,NaN,...,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,5 - Parecer cardiológico,NaN


## Retirando a coluna *ID*, *Convenio* e *SEXO*

In [64]:
df_copy = df.copy()
df_copy.drop(columns=['ID', 'Convenio'], inplace=True)
df_copy.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
0,5.0,51,19.0,11/05/06,30/03/06,0.12,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,3.5,50,14.0,25/05/05,19/05/05,0.02,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia
2,0.0,0,NaN,12/06/01,08/05/05,-4.05,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN
3,8.1,65,19.0,15/10/09,21/04/09,0.50,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN
4,40.0,151,18.0,14/01/08,14/08/95,12.89,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,5 - Parecer cardiológico,NaN


In [104]:
df_copy['SEXO'].isna().sum()

np.int64(0)

In [108]:
df_copy['SEXO'].value_counts()

SEXO
M    6496
F    4527
I     655
Name: count, dtype: int64

In [107]:
dicionario_sexo = {
    'M': 'M',
    'F': 'F',
    'masculino': 'M',
    'Masculino': 'M',     
    'Feminino': 'F',    
    'Indeterminado': 'I'
}

# Aplicando o mapeamento
df_copy['SEXO'] = df_copy['SEXO'].map(dicionario_sexo)

## Tratando a coluna *Idade*

In [65]:
def limpar_data(val):
    try:
        # Tenta converter formato dia/mês/ano
        return pd.to_datetime(val, format='%d/%m/%y')
    except:
        try:
            # Se falhar, tenta converter número serial do Excel
            # (Base do Excel é aprox 30/12/1899)
            return pd.to_datetime(float(val), unit='D', origin='1899-12-30')
        except:
            return pd.NaT
    
df_copy['DN'] = df_copy['DN'].apply(limpar_data)
df_copy['Atendimento'] = df_copy['Atendimento'].apply(limpar_data)
df_copy.head()


,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
0,5.0,51,19.0,2006-05-11,2006-03-30,0.12,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,3.5,50,14.0,2005-05-25,2005-05-19,0.02,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia
2,0.0,0,NaN,2001-06-12,2005-05-08,-4.05,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN
3,8.1,65,19.0,2009-10-15,2009-04-21,0.50,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN
4,40.0,151,18.0,2008-01-14,1995-08-14,12.89,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,5 - Parecer cardiológico,NaN


In [66]:
df_copy['IDADE'] = (df_copy['Atendimento'] - df_copy['DN']).dt.days / 365

df_copy.loc[df_copy['IDADE'] < 0, 'IDADE'] = np.nan

# Comparação
print("Negativos antes:", (df_copy['IDADE'] < 0).sum())
print("Negativos depois:", (df_copy['IDADE'] < 0).sum())
print("Valores recuperados/corrigidos:", df_copy['IDADE'].notnull().sum())

Negativos antes: 0
Negativos depois: 0
Valores recuperados/corrigidos: 11246


## Tratando a coluna *NORMAL X ANORMAL*

In [4]:
df_copy[(df_copy['NORMAL X ANORMAL'].isna()) & (df_copy['SOPRO'] != 'ausente') & (df_copy['SOPRO'].notna())]

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,MOTIVO1,MOTIVO2
1353,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,NaN,NaN,Contínuo,NaN,NaN,NaN,NaN,NaN
1730,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,NaN,NaN,Sistólico,NaN,NaN,NaN,NaN,NaN
12589,5.3,60,15.0,02/12/08,13/09/08,0.23,Normais,NaN,NaN,Não Calculado,NaN,Normal,Sistólico,100,Assintomático,NaN,6 - Suspeita de cardiopatia,6 - Sopro
12615,0.0,0,NaN,12/05/08,18/06/99,9.24,Normais,100.0,60.0,Não Calculado,NaN,Normal,Sistólico,90,Assintomático,NaN,1 - Cardiopatia já estabelecida,1 - Cardiopatia congenica
12687,0.0,0,NaN,16/06/09,10/03/04,5.47,Normais,NaN,NaN,Não Calculado,NaN,Normal,Sistólico,90,Assintomático,NaN,6 - Suspeita de cardiopatia,Outro
12747,46.0,162,18.0,04/08/03,08/04/89,14.86,Normais,90.0,60.0,Não Calculado,NaN,Normal,Sistólico,90,Assintomático,NaN,2 - Check-up,NaN
12757,20.0,0,NaN,01/07/03,17/12/96,6.78,Normais,90.0,60.0,Não Calculado,NaN,Normal,Sistólico,100,Assintomático,NaN,6 - Suspeita de cardiopatia,6 - Sopro
12766,24.0,118,17.0,17/02/09,14/09/03,5.63,Normais,100.0,60.0,Não Calculado,NaN,Normal,Sistólico,78,Assintomático,NaN,6 - Suspeita de cardiopatia,6 - Sopro
12854,30.0,0,NaN,22/08/06,NaN,NaN,Normais,140.0,60.0,#VALUE!,NaN,Normal,Sistólico,120,Dispneia,Cianose,5 - Parecer cardiológico,Outro


In [67]:
df_copy['NORMAL X ANORMAL'].isna().sum()

np.int64(1168)

In [68]:
df_copy.dropna(subset=['NORMAL X ANORMAL'], inplace=True)

In [69]:
df_copy.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO1,MOTIVO2
0,5.0,51,19.0,2006-05-11,2006-03-30,0.115068,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,6 - Suspeita de cardiopatia,6 - Palpitação/taquicardia/arritmia
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016438,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,6 - Suspeita de cardiopatia,6 - Dispnéia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,2 - Check-up,NaN
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484932,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,5 - Parecer cardiológico,NaN
4,40.0,151,18.0,2008-01-14,1995-08-14,12.427397,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,5 - Parecer cardiológico,NaN


## Tratando coluna *Pulsos*

In [72]:
df_copy['PULSOS'].isna().sum()

np.int64(26)

In [33]:
df_copy[(df_copy['PULSOS'].isna()) & (df_copy['NORMAL X ANORMAL'] == 'Anormal')]

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA,MOTIVO
57,32.0,0,NaN,06/08/03,20/10/93,10.16,NaN,100.0,60.0,Não Calculado,Anormal,Normal,Sistólico,NaN,outro,alterações de pulso/perfusão
65,7.1,67,16.0,38185,38035,0.43,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,100,,sopro
84,0.0,0,NaN,15/04/09,27/02/09,0.13,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,200,,sopro
161,5.0,0,NaN,12/03/08,04/02/08,0.11,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,164,dispneia,outro
167,0.0,0,NaN,08/01/04,04/12/03,0.10,NaN,NaN,NaN,Não Calculado,Anormal,Normal,sistólico,152,"cianose,dispneia",sopro
268,0.0,62,0.0,07/03/06,15/09/05,0.49,NaN,NaN,NaN,Não Calculado,Anormal,Hiperfonética,ausente,100,,sopro
418,0.0,0,NaN,22/09/08,09/06/07,1.34,NaN,NaN,NaN,Não Calculado,Anormal,Única,sistólico,160,,cirurgia
443,28.0,138,15.0,28/09/07,14/09/97,10.41,NaN,130.0,70.0,HAS-2 PAS,Anormal,Normal,sistólico,84,assintomático,cardiopatia congenica
450,37.0,151,16.0,25/10/04,08/05/93,11.90,NaN,110.0,70.0,Normal,Anormal,Normal,sistólico,80,,sopro
536,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,Anormal,Hiperfonética,Sistólico,NaN,,NaN


In [71]:
lista_indices = [
    12562, 12578, 12581, 12590, 12607, 12616, 12634, 12636, 
    12662, 12667, 12688, 12708, 12738, 12748, 12758, 12763, 
    12767, 12782, 12803, 12833, 12839, 12855
]

df_copy.drop(index=lista_indices, inplace=True)

In [10]:
df_copy['PULSOS'].value_counts()

PULSOS
Normais                11531
Amplos                    52
Femorais diminuidos       39
Outro                     36
Diminuídos                18
NORMAIS                    2
AMPLOS                     1
Name: count, dtype: int64

In [73]:
df_copy['PULSOS'] = df_copy['PULSOS'].str.strip().str.capitalize()

df_copy['PULSOS'] = df_copy['PULSOS'].fillna('Normais')

print(df_copy['PULSOS'].value_counts())

PULSOS
Normais                11537
Amplos                    53
Femorais diminuidos       39
Outro                     36
Diminuídos                18
Name: count, dtype: int64


## Tratando coluna *Motivo 1 e 2*

In [76]:
df_copy['MOTIVO'].isna().sum()

np.int64(265)

In [12]:
df_copy['MOTIVO1'].value_counts()

MOTIVO1
5 - Parecer cardiológico           5352
6 - Suspeita de cardiopatia        4285
1 - Cardiopatia já estabelecida    1158
2 - Check-up                        701
7 - Outro                           322
Name: count, dtype: int64

In [25]:
df_copy['MOTIVO2'].isna().sum()

np.int64(3496)

In [14]:
df_copy['MOTIVO2'].value_counts()

MOTIVO2
5 - Cirurgia                           2940
6 - Sopro                              2349
1 - Cardiopatia congenica              1041
Outro                                   824
5 - Atividade física                    652
6 - Palpitação/taquicardia/arritmia     415
6 - Dor precordial                      392
6 - HAS/dislipidemia/obesidade          240
6 - Dispnéia                            226
6 - Cianose                             145
1 - Cardiopatia adquirida                92
6 - Cardiopatia na familia               35
5 - Uso de cisaprida                      9
6 - Cansaço                               7
6 - Alterações de pulso/perfusão          5
6 - Cianose e dispnéia                    5
Name: count, dtype: int64

In [74]:
def limpar_motivo(valor):
    if pd.isna(valor):
        return np.nan
    
    texto = str(valor)

    texto_limpo = re.sub(r'^\d+\s*-\s*', '', texto)

    return texto_limpo.strip().lower()

df_copy['MOTIVO1'] = df_copy['MOTIVO1'].apply(limpar_motivo)
df_copy['MOTIVO2'] = df_copy['MOTIVO2'].apply(limpar_motivo)

df_copy['MOTIVO'] = df_copy['MOTIVO2'].fillna(df_copy['MOTIVO1'])

df_copy.drop(columns=['MOTIVO1', 'MOTIVO2'], inplace=True)
df_copy.head()

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,HDA 1,HDA2,SEXO,MOTIVO
0,5.0,51,19.0,2006-05-11,2006-03-30,0.115068,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,Palpitacao,NaN,M,palpitação/taquicardia/arritmia
1,3.5,50,14.0,2005-05-25,2005-05-19,0.016438,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,Dispneia,NaN,M,dispnéia
2,0.0,0,NaN,2001-06-12,2005-05-08,NaN,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,Assintomático,NaN,M,check-up
3,8.1,65,19.0,2009-10-15,2009-04-21,0.484932,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,Assintomático,NaN,M,parecer cardiológico
4,40.0,151,18.0,2008-01-14,1995-08-14,12.427397,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,Dor precordial,NaN,M,parecer cardiológico


## Tratando colunas *HDA 1 e 2*

In [17]:
df_copy['HDA 1'].isna().sum()

np.int64(4270)

In [19]:
df_copy['HDA 1'].value_counts()

HDA 1
Assintomático      6561
Dispneia            541
Dor precordial      508
Palpitacao          365
Cianose             260
Desmaio/tontura     157
Outro               124
Ganho de peso        87
Name: count, dtype: int64

In [18]:
df_copy['HDA2'].isna().sum()

np.int64(12466)

In [20]:
df_copy['HDA2'].value_counts()

HDA2
Dispneia           83
Palpitacao         83
Dor precordial     65
Cianose            65
Desmaio/tontura    50
Outro              41
Ganho de peso      19
Assintomático       1
Name: count, dtype: int64

In [77]:
def unir_sintomas(row):
    sintomas = set()

    if pd.notna(row['HDA 1']):
        sintoma1 = str(row['HDA 1']).strip().lower()
        if sintoma1 not in ['', 'nan']:
            sintomas.add(sintoma1)

    if pd.notna(row['HDA2']):
        sintoma2 = str(row['HDA2']).strip().lower()
        if sintoma2 not in ['', 'nan']:
            sintomas.add(sintoma2)

    return ",".join(sintomas) 

df_copy['HDA'] = df_copy.apply(unir_sintomas, axis=1)

df_copy.drop(columns=['HDA 1', 'HDA2'], inplace=True)

print("Coluna unida:")
print(df_copy['HDA'].head())

Coluna unida:
0        palpitacao
1          dispneia
2     assintomático
3     assintomático
4    dor precordial
Name: HDA, dtype: object


In [78]:
df_copy['HDA'].value_counts()

HDA
assintomático                     6532
                                  3118
dor precordial                     406
dispneia                           400
palpitacao                         306
cianose                            197
desmaio/tontura                    124
outro                              123
ganho de peso                       79
cianose,dispneia                    75
palpitacao,dor precordial           71
palpitacao,dispneia                 41
dispneia,dor precordial             32
dor precordial,dispneia             30
cianose,desmaio/tontura             27
outro,dispneia                      16
palpitacao,desmaio/tontura          15
desmaio/tontura,dor precordial      15
ganho de peso,dispneia              14
desmaio/tontura,dispneia            12
desmaio/tontura,outro                8
palpitacao,cianose                   8
cianose,dor precordial               7
ganho de peso,dor precordial         6
outro,dor precordial                 5
cianose,outro        

In [ ]:
df_copy.head()

## Tratando coluna *PPA*

In [ ]:
df_copy['PPA'].isna().sum()

In [ ]:
df_copy['PPA'].value_counts()

In [ ]:
def atualizar_ppa_dataset(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo)

    bp_ref = {
        1:  {'M': {'S90': 100, 'S95': 104, 'D90': 53, 'D95': 58}, 'F': {'S90': 100, 'S95': 105, 'D90': 55, 'D95': 59}},
        2:  {'M': {'S90': 104, 'S95': 108, 'D90': 58, 'D95': 62}, 'F': {'S90': 104, 'S95': 108, 'D90': 60, 'D95': 65}},
        3:  {'M': {'S90': 107, 'S95': 110, 'D90': 62, 'D95': 66}, 'F': {'S90': 104, 'S95': 108, 'D90': 64, 'D95': 68}},
        4:  {'M': {'S90': 109, 'S95': 112, 'D90': 66, 'D95': 70}, 'F': {'S90': 106, 'S95': 110, 'D90': 67, 'D95': 71}},
        5:  {'M': {'S90': 110, 'S95': 114, 'D90': 69, 'D95': 73}, 'F': {'S90': 107, 'S95': 111, 'D90': 69, 'D95': 73}},
        6:  {'M': {'S90': 111, 'S95': 115, 'D90': 71, 'D95': 75}, 'F': {'S90': 109, 'S95': 113, 'D90': 70, 'D95': 74}},
        7:  {'M': {'S90': 113, 'S95': 117, 'D90': 73, 'D95': 77}, 'F': {'S90': 111, 'S95': 115, 'D90': 72, 'D95': 76}},
        8:  {'M': {'S90': 114, 'S95': 118, 'D90': 74, 'D95': 79}, 'F': {'S90': 113, 'S95': 116, 'D90': 73, 'D95': 77}},
        9:  {'M': {'S90': 115, 'S95': 119, 'D90': 76, 'D95': 80}, 'F': {'S90': 114, 'S95': 118, 'D90': 74, 'D95': 78}},
        10: {'M': {'S90': 117, 'S95': 121, 'D90': 76, 'D95': 81}, 'F': {'S90': 116, 'S95': 120, 'D90': 75, 'D95': 79}},
        11: {'M': {'S90': 119, 'S95': 123, 'D90': 76, 'D95': 81}, 'F': {'S90': 118, 'S95': 122, 'D90': 76, 'D95': 80}},
        12: {'M': {'S90': 121, 'S95': 125, 'D90': 77, 'D95': 82}, 'F': {'S90': 120, 'S95': 124, 'D90': 77, 'D95': 81}},
        13: {'M': {'S90': 124, 'S95': 128, 'D90': 77, 'D95': 82}, 'F': {'S90': 122, 'S95': 126, 'D90': 78, 'D95': 82}},
        14: {'M': {'S90': 126, 'S95': 130, 'D90': 78, 'D95': 83}, 'F': {'S90': 124, 'S95': 127, 'D90': 79, 'D95': 83}},
        15: {'M': {'S90': 129, 'S95': 133, 'D90': 79, 'D95': 84}, 'F': {'S90': 125, 'S95': 129, 'D90': 80, 'D95': 84}},
        16: {'M': {'S90': 131, 'S95': 135, 'D90': 81, 'D95': 86}, 'F': {'S90': 126, 'S95': 130, 'D90': 81, 'D95': 85}},
        17: {'M': {'S90': 134, 'S95': 138, 'D90': 83, 'D95': 87}, 'F': {'S90': 126, 'S95': 130, 'D90': 81, 'D95': 85}}
    }

    def calcular_ppa(row):
        if pd.isna(row['PA SISTOLICA']) or pd.isna(row['PA DIASTOLICA']) or pd.isna(row['IDADE']):
            return row['PPA'] 

        idade = int(row['IDADE'])
        
        if idade < 1:
            return 'Requer Tabela Infantil' 
        
        if idade > 17:
            idade = 17

        # Padroniza Sexo
        sexo = str(row['SEXO']).strip().upper()
        if sexo in ['M', 'MASCULINO', 'BOY']:
            s_key = 'M'
        else:
            s_key = 'F'

        limites = bp_ref[idade][s_key]
        pas = row['PA SISTOLICA']
        pad = row['PA DIASTOLICA']

        if pas >= limites['S95'] or pad >= limites['D95']:
            return 'Hipertensão'
        elif pas >= limites['S90'] or pad >= limites['D90'] or pas >= 120 or pad >= 80:
            return 'Elevada'
        else:
            return 'Normal'

    # 4. Aplica a função
    print("Calculando PPA...")
    df['PPA_Calculada'] = df.apply(calcular_ppa, axis=1)

    mask_update = (df['PPA'].isna()) | (df['PPA'] == 'Não Calculado')
    df.loc[mask_update, 'PPA'] = df.loc[mask_update, 'PPA_Calculada']

    # Remove a coluna temporária
    df.drop(columns=['PPA_Calculada'], inplace=True)
    
    # Exibe estatísticas
    n_nans = df['PPA'].isna().sum() + (df['PPA'] == 'Não Calculado').sum()
    print(f"Processo concluído. NaNs restantes em PPA: {n_nans}")

    # 6. Excluir colunas de Pressão (Opcional - Descomente para ativar)
    colunas_remover = ['PA SISTOLICA', 'PA DIASTOLICA']
    df.drop(columns=colunas_remover, inplace=True)
    print(f"Colunas {colunas_remover} removidas.")

    return df

# Para usar:
df_final = atualizar_ppa_dataset(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_cleaned.csv')
df_final.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_cleaned.csv', index=False)

Calculando PPA...
Processo concluído. NaNs restantes em PPA: 6204
Colunas ['PA SISTOLICA', 'PA DIASTOLICA'] removidas.


In [119]:
df_final.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_cleaned.csv', index=False)

## Traatando a coluna *B2*

In [79]:
df_copy['B2'].isna().sum()

np.int64(9)

In [38]:
df_copy['B2'].value_counts()

B2
Normal           11077
Hiperfonética      295
Desdob fixo        149
Única               77
Outro               76
Name: count, dtype: int64

In [80]:
df_copy['B2'] = df_copy['B2'].fillna('Normal')

## Tratando coluna *SOPRO*

In [84]:
df_copy['SOPRO'].isna().sum()

np.int64(0)

In [40]:
df_copy['SOPRO'].value_counts()

SOPRO
ausente                   7260
Sistólico                 3640
sistólico                  723
contínuo                    28
Contínuo                    19
diastólico                   9
Sistolico e diastólico       3
Name: count, dtype: int64

In [82]:
df_copy[df_copy['SOPRO'].isna()]

,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
1951,0.0,0,NaN,2009-04-08,2003-08-21,5.635616,Outro,NaN,NaN,Não Calculado,Normal,Normal,NaN,NaN,F,cirurgia,


In [83]:
df_copy.dropna(subset=['SOPRO'], inplace=True)

In [ ]:
print(len(df_copy))

## Retirar linhas ou colunas com mais de 50% de dados faltantes

In [112]:
(df_copy.isna().mean() * 100).sort_values(ascending=False)

PA DIASTOLICA       56.165439
PA SISTOLICA        56.079808
MOTIVO               2.269224
PPA                  1.858195
Altura               0.000000
IMC                  0.000000
Peso                 0.000000
IDADE                0.000000
PULSOS               0.000000
B2                   0.000000
NORMAL X ANORMAL     0.000000
SOPRO                0.000000
FC                   0.000000
SEXO                 0.000000
HDA                  0.000000
dtype: float64

In [ ]:
df_copy.drop(columns=['PA DIASTOLICA', 'PA SISTOLICA'], inplace=True)

In [110]:
(df_copy.isna().mean() * 100).sort_values(ascending=False)

PA DIASTOLICA       56.165439
PA SISTOLICA        56.079808
MOTIVO               2.269224
PPA                  1.858195
Altura               0.000000
IMC                  0.000000
Peso                 0.000000
IDADE                0.000000
PULSOS               0.000000
B2                   0.000000
NORMAL X ANORMAL     0.000000
SOPRO                0.000000
FC                   0.000000
SEXO                 0.000000
HDA                  0.000000
dtype: float64

In [87]:
# 1. Calcula a porcentagem de nulos para cada linha
percentual_nulos_linha = df_copy.isna().mean(axis=1)

# 2. Filtra apenas as linhas com mais de 50% (0.5) de nulos
linhas_vazias = df_copy[percentual_nulos_linha > 0.5]

# Mostra quantas linhas foram encontradas
print(f"Foram encontradas {len(linhas_vazias)} linhas com mais de 50% de dados faltantes.")

# Visualiza essas linhas (mostrando apenas as primeiras 5 para não poluir a tela)
linhas_vazias.head()

Foram encontradas 0 linhas com mais de 50% de dados faltantes.


,Peso,Altura,IMC,Atendimento,DN,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA


In [88]:
# 1. Identifica os índices das linhas com mais de 50% (0.5) de nulos
linhas_para_remover = df_copy[df_copy.isna().mean(axis=1) > 0.5].index

# 2. Remove as linhas usando .drop()
df_copy.drop(linhas_para_remover, inplace=True)

# 3. Feedback para você saber quantas foram apagadas
print(f"Foram removidas {len(linhas_para_remover)} linhas.")

Foram removidas 0 linhas.


In [89]:
df_copy.drop(columns=['DN', 'Atendimento'], inplace=True)

In [ ]:
df_copy.head()

In [109]:
df_copy.to_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_cleaned.csv', index=False)

## Tratando as colunas numéricas

In [91]:
cols_biologicas = ['Peso', 'Altura', 'IDADE']

for col in cols_biologicas:
    if col in df_copy.columns:
        qtd_erros = df_copy[df_copy[col] <= 0].shape[0]
        if qtd_erros > 0:
            print(f"Coluna '{col}': {qtd_erros} valores inválidos (<=0) transformados em NaN.")
            
            df_copy.loc[df_copy[col] <= 0, col] = np.nan


mask_imc = df_copy['Peso'].notnull() & df_copy['Altura'].notnull() & df_copy['IMC'].isnull()
df_copy.loc[mask_imc, 'IMC'] = df_copy.loc[mask_imc, 'Peso'] / ((df_copy.loc[mask_imc, 'Altura']/100) ** 2)

for col in cols_biologicas + ['IMC']:
    if col in df_copy.columns:
        mediana = df_copy[col].median()
        df_copy[col] = df_copy[col].fillna(mediana)

print("\nVerificação Final de Nulos:")
print(df_copy[cols_biologicas].isnull().sum())

Coluna 'Peso': 1793 valores inválidos (<=0) transformados em NaN.
Coluna 'Altura': 3611 valores inválidos (<=0) transformados em NaN.
Coluna 'IDADE': 27 valores inválidos (<=0) transformados em NaN.

Verificação Final de Nulos:
Peso      0
Altura    0
IDADE     0
dtype: int64


In [92]:
df_copy.head()

,Peso,Altura,IMC,IDADE,PULSOS,PA SISTOLICA,PA DIASTOLICA,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,SEXO,MOTIVO,HDA
0,5.0,51.0,19.0,0.115068,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,112,M,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50.0,14.0,0.016438,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,128,M,dispnéia,dispneia
2,16.0,99.0,17.0,3.823288,Normais,NaN,NaN,Não Calculado,Anormal,Normal,Sistólico,88,M,check-up,assintomático
3,8.1,65.0,19.0,0.484932,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,92,M,parecer cardiológico,assintomático
4,40.0,151.0,18.0,12.427397,Normais,NaN,NaN,Não Calculado,Anormal,Normal,ausente,96,M,parecer cardiológico,dor precordial


In [93]:
df_copy.isna().sum()

Peso                   0
Altura                 0
IMC                    0
IDADE                  0
PULSOS                 0
PA SISTOLICA        6551
PA DIASTOLICA       6561
PPA                  217
NORMAL X ANORMAL       0
B2                     0
SOPRO                  0
FC                   696
SEXO                   4
MOTIVO               265
HDA                    0
dtype: int64

## Tratando *FC*

In [94]:
def clean_fc(val):
    if pd.isna(val):
        return np.nan
    
    val_str = str(val).strip()
    
    if '-' in val_str:
        try:
            parts = val_str.split('-')
            return np.mean([float(p) for p in parts])
        except:
            return np.nan
    if ' a ' in val_str:
        try:
            parts = val_str.split(' a ')
            return np.mean([float(p) for p in parts])
        except:
            return np.nan

    try:
        num = float(val_str)
    except ValueError:
        return np.nan

    if num > 300: 
        if num < 3000: # Provável erro de x10 (ex: 1120 -> 112)
             num = num / 10
        elif num < 30000: # Provável erro de x100 (ex: 9288 -> 92.88)
             num = num / 100
        else:
             return np.nan # Dado irrecuperável

    if num < 30: 
        return np.nan

    return round(num)

df_copy['FC'] = df_copy['FC'].apply(clean_fc)

In [95]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11682 entries, 0 to 12872
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Peso              11682 non-null  float64
 1   Altura            11682 non-null  float64
 2   IMC               11682 non-null  float64
 3   IDADE             11682 non-null  float64
 4   PULSOS            11682 non-null  object 
 5   PA SISTOLICA      5131 non-null   float64
 6   PA DIASTOLICA     5121 non-null   float64
 7   PPA               11465 non-null  object 
 8   NORMAL X ANORMAL  11682 non-null  object 
 9   B2                11682 non-null  object 
 10  SOPRO             11682 non-null  object 
 11  FC                10980 non-null  float64
 12  SEXO              11678 non-null  object 
 13  MOTIVO            11417 non-null  object 
 14  HDA               11682 non-null  object 
dtypes: float64(7), object(8)
memory usage: 1.4+ MB


In [96]:
df_copy['Age_Int'] = df_copy['IDADE'].apply(lambda x: int(x) if pd.notnull(x) else x)

df_copy['FC'] = df_copy['FC'].fillna(df_copy.groupby('Age_Int')['FC'].transform('median'))

df_copy['FC'] = df_copy['FC'].fillna(df_copy['FC'].median())

df_copy.drop(columns=['Age_Int'], inplace=True)

In [97]:
df_copy['FC'].isna().sum()

np.int64(0)

In [98]:
df_copy['FC'].describe()

count    11682.000000
mean        95.269432
std         17.795335
min         43.000000
25%         80.000000
50%         92.000000
75%        100.000000
max        300.000000
Name: FC, dtype: float64

## Tratamento final de valores ausentes

In [120]:
df_cleaned = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\MD\estudo_caso\data\UCMF_cleaned.csv')
df_cleaned.head()

,Peso,Altura,IMC,IDADE,PULSOS,PPA,NORMAL X ANORMAL,B2,SOPRO,FC,MOTIVO,HDA
0,5.0,51.0,19.0,0.115068,Normais,Não Calculado,Anormal,Normal,Sistólico,112.0,palpitação/taquicardia/arritmia,palpitacao
1,3.5,50.0,14.0,0.016438,Normais,Não Calculado,Anormal,Normal,ausente,128.0,dispnéia,dispneia
2,16.0,99.0,17.0,3.823288,Normais,Não Calculado,Anormal,Normal,Sistólico,88.0,check-up,assintomático
3,8.1,65.0,19.0,0.484932,Normais,Não Calculado,Anormal,Normal,ausente,92.0,parecer cardiológico,assintomático
4,40.0,151.0,18.0,12.427397,Normais,Não Calculado,Anormal,Normal,ausente,96.0,parecer cardiológico,dor precordial


In [125]:
df_cleaned.isna().sum()

Peso                0
Altura              0
IMC                 0
IDADE               0
PULSOS              0
PPA                 0
NORMAL X ANORMAL    0
B2                  0
SOPRO               0
FC                  0
MOTIVO              0
HDA                 0
dtype: int64

In [122]:
df_cleaned['HDA'] = df_cleaned['HDA'].fillna('Não Informado')

In [124]:
def preencher_motivo(row):
    if pd.notna(row['MOTIVO']):
        return row['MOTIVO']
    
    hda = str(row['HDA']).lower()
    
    if 'dispneia' in hda:
        return 'dispnéia'  
    elif 'dor precordial' in hda:
        return 'dor precordial'
    elif 'palpitacao' in hda or 'palpitação' in hda:
        return 'palpitação/taquicardia/arritmia'
    elif 'cianose' in hda:
        return 'cianose'
    elif 'assintomático' in hda:
        return 'parecer cardiológico' 
    else:
        return 'Não Informado'

df_cleaned['MOTIVO'] = df_cleaned.apply(preencher_motivo, axis=1)

print("Valores ausentes restantes em MOTIVO:", df_cleaned['MOTIVO'].isnull().sum())
print(df_cleaned['MOTIVO'].value_counts())

Valores ausentes restantes em MOTIVO: 0
MOTIVO
cirurgia                           2844
sopro                              2284
parecer cardiológico               1744
cardiopatia congenica              1017
outro                               783
check-up                            653
atividade física                    586
palpitação/taquicardia/arritmia     400
dor precordial                      375
dispnéia                            229
has/dislipidemia/obesidade          226
Não Informado                       192
cianose                             141
cardiopatia adquirida                90
suspeita de cardiopatia              59
cardiopatia na familia               32
uso de cisaprida                      9
cansaço                               6
alterações de pulso/perfusão          5
cianose e dispnéia                    3
Name: count, dtype: int64
